In [3]:
!uv sync
%load_ext autoreload 
%autoreload 2  
# imported functions are re-imported if they are changed

Resolved 151 packages in 0.41ms
Audited 148 packages in 0.10ms
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from pyorbital.orbital import Orbital
import datetime as dt
from functools import lru_cache

@lru_cache(maxsize=2)
def get_satellite_orbital(satellite_name):
    
    possible_options = ["TERRA", "AQUA"]
    if satellite_name not in possible_options:
        raise ValueError(f"Satellite name should be one of {possible_options}")

    return Orbital(satellite_name)

In [ ]:
def get_modis_overpass_time(satellite_name:str, date:str, lat:float, lon:float, altitude=0.3, max_angle=35):
    import pandas as pd 

    satellite = get_satellite_orbital(satellite_name)

    dtobj = dt.datetime.strptime(date, "%Y-%m-%d")

    passes = satellite.get_next_passes(dtobj, 24, lon, lat, altitude, horizon=max_angle)
    
    # get overhead time - compared against https://oceandata.sci.gsfc.nasa.gov/overpass_pred/
    overhead_time = pd.to_datetime([p[2] for p in passes])

    df = pd.DataFrame(passes, columns=["rise_time", "fall_time", "max_elevation_time"])

    df['satellite'] = satellite_name
    df['request_date'] = pd.to_datetime(dtobj)
    df['lat'] = lat
    df['lon'] = lon
    df['altitude'] = altitude
    df['max_angle'] = max_angle
    df['night_pass'] = [~is_daytime(t, lat, lon) for t in overhead_time]
    
    return df


def is_daytime(time, lat, lon):
    """Returns True if it's daytime at the given location, False otherwise."""

    from datetime import datetime, timezone
    from pyorbital.astronomy import sun_zenith_angle

    zenith_angle = sun_zenith_angle(time, float(lon), float(lat))
    
    return zenith_angle < 90  # Daytime if the sun is above the horizon

In [151]:
get_modis_overpass_time("AQUA", "2025-02-12", lat=-34, lon=20)

,rise_time,fall_time,max_elevation_time,satellite,request_date,lat,lon,altitude,max_angle,night_pass
0,2025-02-12 01:11:47.435430,2025-02-12 01:14:41.318492,2025-02-12 01:13:14.112770,AQUA,2025-02-12,-34,20,0.3,35,True
1,2025-02-12 13:13:01.668337,2025-02-12 13:16:31.835558,2025-02-12 13:14:47.122990,AQUA,2025-02-12,-34,20,0.3,35,False
